<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [1]:
!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

path_input_fan_id_00 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "fan", "id_00")
path_input_fan_id_02 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "fan", "id_02")
path_input_fan_id_04 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "fan", "id_04")
path_input_fan_id_06 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "fan", "id_06")

path_input_pump_id_00 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "pump", "id_00")
path_input_pump_id_02 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "pump", "id_02")
path_input_pump_id_04 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "pump", "id_04")
path_input_pump_id_06 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "pump", "id_06")

path_input_slider_id_00 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "slider", "id_00")
path_input_slider_id_02 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "slider", "id_02")
path_input_slider_id_04 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "slider", "id_04")
path_input_slider_id_06 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "slider", "id_06")

path_input_valve_id_00 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "valve", "id_00")
path_input_valve_id_02 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "valve", "id_02")
path_input_valve_id_04 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "valve", "id_04")
path_input_valve_id_06 = os.path.join(os.getcwd(),  "..", "..", "..", "MIMII", "valve", "id_06")

In [4]:
import torch
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) > 0, "Not enough GPU hardware devices available"
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

AssertionError: Not enough GPU hardware devices available

In [5]:
import sys
sys.path.append("..")

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Reading Data Files:

In [7]:
from mtsa import calculate_aucroc

In [8]:
from mtsa import files_train_test_split
#path_input_1 = '/data/MIMII/fan/id_00/'
X_train, X_test, y_train, y_test = files_train_test_split(path_input_fan_id_06)
#if(len(y_train) == 0): 
#   X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
print(f"X_train: {X_train}")
print(f"y_train: {y_train}")

X_train: ['/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000555.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000112.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000134.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000036.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000788.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000422.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000752.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000453.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000685.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000772.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000170.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000682.wav'
 '/data/joao/mtsa/examples/../../../MIMII/fan/id_06/normal/00000517.wav'
 '/data/joao/mtsa/examples/../../../MIMII/

### MFFCMix Model:

In [9]:
#from mtsa import MFCCMix
#model_mfccmix = MFCCMix()
#model_mfccmix.fit(X_train, y_train)
#model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [10]:
#auc = calculate_aucroc(model_mfccmix, X_test, y_test)
#auc

0.8609356895665318

### IForest Model


In [ ]:
from mtsa import IForest

model_isolationforest = IForest()
model_isolationforest.model

ModuleNotFoundError: No module named 'mtsa.models.isolationforest'

In [ ]:
auc = calculate_aucroc(model_isolationforest, X_test, y_test)
auc

### Hitachi Model:

In [11]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

KeyboardInterrupt: 

In [ ]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

### GANF Model:

In [ ]:
from mtsa.models.ganf import GANF
model_GANF = GANF()

In [ ]:
model_GANF.fit(X_train, y_train)
model_GANF.model

In [ ]:
auc = calculate_aucroc(model_GANF, X_test, y_test)
auc

### RANSynCoders Model:

In [ ]:
from mtsa import RANSynCoders
model_ransyncorders = RANSynCoders(is_acoustic_data=True, mono=True, normal_classifier=1, abnormal_classifier=0, synchronize=True)
model_ransyncorders.fit(X_train, y_train)

In [ ]:
auc = calculate_aucroc(model_ransyncorders, X_test, y_test)
auc